In [2]:
import sympy
from sympy import *
import sympy as sym
from sympy.physics.quantum import TensorProduct as tp # produto tensorial
import math
from sympy.physics.quantum import Ket, Bra

# Funções auxiliares

In [200]:
def tr(A):
    '''retorna o traço de uma matriz'''
    d = A.shape[0]
    tr = 0
    for j in range(0,d):
        tr += A[j,j]
    return tr

In [3]:
def cb(n,j):
    '''retorna um vetor da base padrão de C^n'''
    vec = zeros(n,1)
    vec[j] = 1
    return vec

In [4]:
display(cb(2,0), cb(2,1))

Matrix([
[1],
[0]])

Matrix([
[0],
[1]])

In [5]:
display(cb(3,0), cb(3,1))

Matrix([
[1],
[0],
[0]])

Matrix([
[0],
[1],
[0]])

In [6]:
def proj(psi): 
    '''retorna o projetor no vetor psi'''
    d = psi.shape[0]
    P = zeros(d,d)
    for j in range(0,d):
        for k in range(0,d):
            P[j,k] = psi[j]*conjugate(psi[k])
    return P

In [7]:
display(proj(cb(2,0)),proj(cb(2,1)))

Matrix([
[1, 0],
[0, 0]])

Matrix([
[0, 0],
[0, 1]])

In [8]:
def inner_product(v,w):
    d = len(v); ip = 0
    for j in range(0,d):
        ip += conjugate(v[j])*w[j]
    return ip

In [9]:
a,b,c,d = symbols("a b c d"); v = [b,a]; w = [c,d]; inner_product(v,w)

c*conjugate(b) + d*conjugate(a)

In [10]:
def ext_product(psi1, psi2): 
    '''retorna o projetor no vetor psi'''
    d1 = psi1.shape[0]
    d2 = psi2.shape[0]
    P = zeros(d1,d2)
    for j in range(0,d1):
        for k in range(0,d2):
            P[j,k] = psi1[j]*conjugate(psi2[k])
    return P

In [11]:
display(ext_product(cb(2,0),cb(2,0)),ext_product(cb(2,0),cb(2,1)),ext_product(cb(2,1),cb(2,0)),ext_product(cb(2,1),cb(2,1)))

Matrix([
[1, 0],
[0, 0]])

Matrix([
[0, 1],
[0, 0]])

Matrix([
[0, 0],
[1, 0]])

Matrix([
[0, 0],
[0, 1]])

In [12]:
def norm(v):
    v = inner_product(v,v)
    return sqrt(v)

In [13]:
v = [2,2]; norm(v)

2*sqrt(2)

In [14]:
# Outside these functions, initialize: rhos = zeros(ds,ds), s=A,B
#
# da = dim de A
# db = dim de B
#
# rho_AB
#
# rho_B = Tr_A (rho_AB)
#
def ptraceA(da, db, rho):
    rhoB = zeros(db,db)
    for j in range(0, db):
        for k in range(0, db):
            for l in range(0, da):
                rhoB[j,k] += rho[l*db+j,l*db+k]
    return rhoB
#
# rho_A = Tr_B (rho_AB)
#
def ptraceB(da, db, rho):
    rhoA = zeros(da,da)
    for j in range(0, da):
        for k in range(0, da):
            for l in range(0, db):
                rhoA[j,k] += rho[j*db+l,k*db+l]
    return rhoA

# Criando um rho genérico de $n$ níveis

In [168]:
# Retorna um rho genérico simbólico com índices na forma binária
# matriz quadrada
# n = dimensão da coluna
def rho_bin(n):
    rho = [sympy.symbols('rho_{}'.format(format(i, '0{}b'.format(int(math.ceil(log(n**2, 2))))))) for i in range(n**2)]
    A = zeros(n,n)
    l = 0
    for j in range(0,n):
        for k in range(0,n):
                A[j,k] = rho[l]
                l += 1
    return A
# rho = rho_bin(16); rho

In [167]:
# Retorna um rho genérico simbólico
# n = dimensão da coluna
# Retorna um rho genérico simbólico
# matriz quadrada
# n = quantidade de níveis
# symbol: 0 ou outro valor = \rho, 1 = \sigma
def rho_g(n, symbol):
    num_digits = len(str(n**2 - 1))
    A = zeros(n,n)
    l = 0
    if symbol == 1:
        rho = sympy.symbols(['sigma{:0{}}'.format(i, num_digits) for i in range(n**2)])
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    else: 
        rho = sympy.symbols(['rho_{:0{}}'.format(i, num_digits) for i in range(n**2)])
        for j in range(0,n):
            for k in range(0,n):
                    A[j,k] = rho[l]
                    l += 1
    return A

In [79]:
m1 = rho_g(4,0)
m1

Matrix([
[rho_00, rho_01, rho_02, rho_03],
[rho_04, rho_05, rho_06, rho_07],
[rho_08, rho_09, rho_10, rho_11],
[rho_12, rho_13, rho_14, rho_15]])

In [80]:
m2 = rho_g(4, 1)
m2

Matrix([
[sigma00, sigma01, sigma02, sigma03],
[sigma04, sigma05, sigma06, sigma07],
[sigma08, sigma09, sigma10, sigma11],
[sigma12, sigma13, sigma14, sigma15]])

In [83]:
m = m1*m2
m

Matrix([
[rho_00*sigma00 + rho_01*sigma04 + rho_02*sigma08 + rho_03*sigma12, rho_00*sigma01 + rho_01*sigma05 + rho_02*sigma09 + rho_03*sigma13, rho_00*sigma02 + rho_01*sigma06 + rho_02*sigma10 + rho_03*sigma14, rho_00*sigma03 + rho_01*sigma07 + rho_02*sigma11 + rho_03*sigma15],
[rho_04*sigma00 + rho_05*sigma04 + rho_06*sigma08 + rho_07*sigma12, rho_04*sigma01 + rho_05*sigma05 + rho_06*sigma09 + rho_07*sigma13, rho_04*sigma02 + rho_05*sigma06 + rho_06*sigma10 + rho_07*sigma14, rho_04*sigma03 + rho_05*sigma07 + rho_06*sigma11 + rho_07*sigma15],
[rho_08*sigma00 + rho_09*sigma04 + rho_10*sigma08 + rho_11*sigma12, rho_08*sigma01 + rho_09*sigma05 + rho_10*sigma09 + rho_11*sigma13, rho_08*sigma02 + rho_09*sigma06 + rho_10*sigma10 + rho_11*sigma14, rho_08*sigma03 + rho_09*sigma07 + rho_10*sigma11 + rho_11*sigma15],
[rho_12*sigma00 + rho_13*sigma04 + rho_14*sigma08 + rho_15*sigma12, rho_12*sigma01 + rho_13*sigma05 + rho_14*sigma09 + rho_15*sigma13, rho_12*sigma02 + rho_13*sigma06 + rho_14*sigma

In [84]:
m = m2*m1
m

Matrix([
[rho_00*sigma00 + rho_04*sigma01 + rho_08*sigma02 + rho_12*sigma03, rho_01*sigma00 + rho_05*sigma01 + rho_09*sigma02 + rho_13*sigma03, rho_02*sigma00 + rho_06*sigma01 + rho_10*sigma02 + rho_14*sigma03, rho_03*sigma00 + rho_07*sigma01 + rho_11*sigma02 + rho_15*sigma03],
[rho_00*sigma04 + rho_04*sigma05 + rho_08*sigma06 + rho_12*sigma07, rho_01*sigma04 + rho_05*sigma05 + rho_09*sigma06 + rho_13*sigma07, rho_02*sigma04 + rho_06*sigma05 + rho_10*sigma06 + rho_14*sigma07, rho_03*sigma04 + rho_07*sigma05 + rho_11*sigma06 + rho_15*sigma07],
[rho_00*sigma08 + rho_04*sigma09 + rho_08*sigma10 + rho_12*sigma11, rho_01*sigma08 + rho_05*sigma09 + rho_09*sigma10 + rho_13*sigma11, rho_02*sigma08 + rho_06*sigma09 + rho_10*sigma10 + rho_14*sigma11, rho_03*sigma08 + rho_07*sigma09 + rho_11*sigma10 + rho_15*sigma11],
[rho_00*sigma12 + rho_04*sigma13 + rho_08*sigma14 + rho_12*sigma15, rho_01*sigma12 + rho_05*sigma13 + rho_09*sigma14 + rho_13*sigma15, rho_02*sigma12 + rho_06*sigma13 + rho_10*sigma

In [82]:
trace(m)

rho_00*sigma00 + rho_01*sigma04 + rho_02*sigma08 + rho_03*sigma12 + rho_04*sigma01 + rho_05*sigma05 + rho_06*sigma09 + rho_07*sigma13 + rho_08*sigma02 + rho_09*sigma06 + rho_10*sigma10 + rho_11*sigma14 + rho_12*sigma03 + rho_13*sigma07 + rho_14*sigma11 + rho_15*sigma15

In [32]:
rho_0 = rho_g(11); rho_0

Matrix([
[rho_000, rho_001, rho_002, rho_003, rho_004, rho_005, rho_006, rho_007, rho_008, rho_009, rho_010],
[rho_011, rho_012, rho_013, rho_014, rho_015, rho_016, rho_017, rho_018, rho_019, rho_020, rho_021],
[rho_022, rho_023, rho_024, rho_025, rho_026, rho_027, rho_028, rho_029, rho_030, rho_031, rho_032],
[rho_033, rho_034, rho_035, rho_036, rho_037, rho_038, rho_039, rho_040, rho_041, rho_042, rho_043],
[rho_044, rho_045, rho_046, rho_047, rho_048, rho_049, rho_050, rho_051, rho_052, rho_053, rho_054],
[rho_055, rho_056, rho_057, rho_058, rho_059, rho_060, rho_061, rho_062, rho_063, rho_064, rho_065],
[rho_066, rho_067, rho_068, rho_069, rho_070, rho_071, rho_072, rho_073, rho_074, rho_075, rho_076],
[rho_077, rho_078, rho_079, rho_080, rho_081, rho_082, rho_083, rho_084, rho_085, rho_086, rho_087],
[rho_088, rho_089, rho_090, rho_091, rho_092, rho_093, rho_094, rho_095, rho_096, rho_097, rho_098],
[rho_099, rho_100, rho_101, rho_102, rho_103, rho_104, rho_105, rho_106, rho_107, 

In [34]:
trace(rho_0)

rho_000 + rho_012 + rho_024 + rho_036 + rho_048 + rho_060 + rho_072 + rho_084 + rho_096 + rho_108 + rho_120

In [58]:
rho_0 = rho_g(4); rho_0

Matrix([
[rho_00, rho_01, rho_02, rho_03],
[rho_04, rho_05, rho_06, rho_07],
[rho_08, rho_09, rho_10, rho_11],
[rho_12, rho_13, rho_14, rho_15]])

# Função que retorna os elementos do vetor de estado ou rho na notação de braket

In [150]:
# Mandar sequência em binário {0,1}
def psi_ket(seq):
    vec = []
    for digito in seq:
        vec.append(int(digito))
    n = len(vec)
    psi = cb(2, vec[0])
    for j in range(1,n):
        psi = tp(psi,cb(2, vec[j]))
    return psi
#psi_ket('00110')

In [151]:
# Mandar sequência em binário {0,1}
def psi_bra(seq):
    vec = []
    for digito in seq:
        vec.append(int(digito))
    n = len(vec)
    psi = cb(2, vec[0])
    for j in range(1,n):
        psi = tp(psi,cb(2, vec[j]))
    return transpose(psi).as_mutable()
#psi_ket('00110')

In [152]:
# Mandar a sequência na dimensão seq (cada dígito --> d-1)
# Exemplo: dim=4, seq = '2023'
def psi_ket_g(dim, seq):
    vec = []
    for digito in seq:
        vec.append(int(digito))
    n = len(vec)
    psi = cb(dim, vec[0])
    for j in range(n-1):
        psi = tp(psi,cb(dim, vec[j+1]))
    return psi
# psi_ket_g(4, '2023')

In [153]:
# Retorna Psi ou rho na notação ketbra
# Mandou Psi - retorna Psi
# Mandou rho - retorna rho
def m_braket(matrix):
    posicoes = []
    posicoes_bin = []
    val = []
    if isinstance(matrix, sympy.matrices.dense.MutableDenseMatrix):
        n_linhas, n_colunas = matrix.shape
        if n_linhas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    posicoes.append(i)
                    posicoes_bin.append(format(i, f'0{int(math.log(x)/math.log(2))}b'))
            for i in range(len(val)):
                Psi = val[i] * Bra(posicoes_bin[i]) + Psi
            return Psi
        if n_colunas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    posicoes.append(i)
                    posicoes_bin.append(format(i, f'0{int(math.log(x)/math.log(2))}b'))
            for i in range(len(val)):
                Psi = val[i] * Ket(posicoes_bin[i]) + Psi
            return Psi
        else:
            m, n = matrix.shape
            rho = 0
            for i in range(m):
                for j in range(n):
                    if matrix[i, j] != 0:
                        val.append(matrix[i, j])
                        posicoes.append((i, j))
                        posicoes_bin.append((format(i, f'0{int(math.log(m)/math.log(2))}b'),
                                            format(j, f'0{int(math.log(n)/math.log(2))}b')))
            for i in range(len(val)):
                rho = val[i] * (Ket(posicoes_bin[i][0])) * (Bra(posicoes_bin[i][1])) + rho
            return rho

In [154]:
m_braket(rho_0)

rho_00*|00>*<00| + rho_01*|00>*<01| + rho_02*|00>*<10| + rho_03*|00>*<11| + rho_04*|00>*<100| + rho_05*|01>*<00| + rho_06*|01>*<01| + rho_07*|01>*<10| + rho_08*|01>*<11| + rho_09*|01>*<100| + rho_10*|10>*<00| + rho_11*|10>*<01| + rho_12*|10>*<10| + rho_13*|10>*<11| + rho_14*|10>*<100| + rho_15*|11>*<00| + rho_16*|11>*<01| + rho_17*|11>*<10| + rho_18*|11>*<11| + rho_19*|11>*<100| + rho_20*|100>*<00| + rho_21*|100>*<01| + rho_22*|100>*<10| + rho_23*|100>*<11| + rho_24*|100>*<100|

In [194]:
# Retorna Psi ou rho na notação ketbra
# Mandou Psi - retorna Psi
# Mandou rho - retorna rho
def m_braket_g(base, matrix):
    def convert_base(x, base, min_digits):
        digits = '0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ'[:base]
        result = ''
        while x > 0 or len(result) < min_digits:
            x, digit = divmod(x, base)
            result = digits[digit] + result
        return result
    posicoes = []
    posicoes_bin = []
    val = []
    if isinstance(matrix, sympy.matrices.dense.MutableDenseMatrix):
        n_linhas, n_colunas = matrix.shape
        if n_linhas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    posicoes.append(i)
                    posicoes_bin.append(convert_base(i, base, int(math.ceil(math.log(x)/math.log(base)))))
            for i in range(len(val)):
                Psi = val[i] * Bra(posicoes_bin[i]) + Psi
            return simplify(Psi)
        if n_colunas == 1:
            Psi = 0
            x = len(matrix)
            for i in range(x):
                if matrix[i] != 0:
                    val.append(matrix[i])
                    posicoes.append(i)
                    posicoes_bin.append(convert_base(i, base, int(math.ceil(math.log(x)/math.log(base)))))
            for i in range(len(val)):
                Psi = val[i] * Ket(posicoes_bin[i]) + Psi
            return Psi
        else:
            m, n = matrix.shape
            rho = 0
            for i in range(m):
                for j in range(n):
                    if matrix[i, j] != 0:
                        val.append(matrix[i, j])
                        posicoes.append((i, j))
                        posicoes_bin.append((convert_base(i, base, int(math.ceil(math.log(m)/math.log(base)))),\
                                             convert_base(j, base, int(math.ceil(math.log(m)/math.log(base))))))
            for i in range(len(val)):
                rho = val[i] * (Ket(posicoes_bin[i][0])) * (Bra(posicoes_bin[i][1])) + rho
            return rho
#m_braket_g(4, psi_ket_g(4,'2023'))

In [156]:
rho_0 = rho_g(5, 0); rho_0

Matrix([
[rho_00, rho_01, rho_02, rho_03, rho_04],
[rho_05, rho_06, rho_07, rho_08, rho_09],
[rho_10, rho_11, rho_12, rho_13, rho_14],
[rho_15, rho_16, rho_17, rho_18, rho_19],
[rho_20, rho_21, rho_22, rho_23, rho_24]])

In [157]:
m_braket_g(2, rho_0)

rho_00*|000>*<000| + rho_01*|000>*<001| + rho_02*|000>*<010| + rho_03*|000>*<011| + rho_04*|000>*<100| + rho_05*|001>*<000| + rho_06*|001>*<001| + rho_07*|001>*<010| + rho_08*|001>*<011| + rho_09*|001>*<100| + rho_10*|010>*<000| + rho_11*|010>*<001| + rho_12*|010>*<010| + rho_13*|010>*<011| + rho_14*|010>*<100| + rho_15*|011>*<000| + rho_16*|011>*<001| + rho_17*|011>*<010| + rho_18*|011>*<011| + rho_19*|011>*<100| + rho_20*|100>*<000| + rho_21*|100>*<001| + rho_22*|100>*<010| + rho_23*|100>*<011| + rho_24*|100>*<100|

In [158]:
m_braket_g(5, rho_0)

rho_00*|0>*<0| + rho_01*|0>*<1| + rho_02*|0>*<2| + rho_03*|0>*<3| + rho_04*|0>*<4| + rho_05*|1>*<0| + rho_06*|1>*<1| + rho_07*|1>*<2| + rho_08*|1>*<3| + rho_09*|1>*<4| + rho_10*|2>*<0| + rho_11*|2>*<1| + rho_12*|2>*<2| + rho_13*|2>*<3| + rho_14*|2>*<4| + rho_15*|3>*<0| + rho_16*|3>*<1| + rho_17*|3>*<2| + rho_18*|3>*<3| + rho_19*|3>*<4| + rho_20*|4>*<0| + rho_21*|4>*<1| + rho_22*|4>*<2| + rho_23*|4>*<3| + rho_24*|4>*<4|

## Exemplos

### Exemplo 1

In [159]:
psi = psi_ket('000000') + 1j*psi_ket('000001') - 1j*psi_ket('000010') + psi_ket('000011')
psi = psi/norm(psi)
transpose(psi)

Matrix([[0.5, 0.5*I, -0.5*I, 0.5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [160]:
psik = m_braket(psi)
psik

0.5*|000000> + 0.5*I*|000001> - 0.5*I*|000010> + 0.5*|000011>

In [161]:
rho = proj(psi)
rho

Matrix([
[   0.25, -0.25*I, 0.25*I,    0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[ 0.25*I,    0.25,  -0.25,  0.25*I, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[-0.25*I,   -0.25,   0.25, -0.25*I, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[   0.25, -0.25*I, 0.25*I,    0.25, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
[      0,       0,      0,       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [162]:
rhokb = m_braket(proj(psi))
rhokb

0.25*|000000>*<000000| - 0.25*I*|000000>*<000001| + 0.25*I*|000000>*<000010| + 0.25*|000000>*<000011| + 0.25*I*|000001>*<000000| + 0.25*|000001>*<000001| - 0.25*|000001>*<000010| + 0.25*I*|000001>*<000011| - 0.25*I*|000010>*<000000| - 0.25*|000010>*<000001| + 0.25*|000010>*<000010| - 0.25*I*|000010>*<000011| + 0.25*|000011>*<000000| - 0.25*I*|000011>*<000001| + 0.25*I*|000011>*<000010| + 0.25*|000011>*<000011|

### Exemplo 2

In [170]:
rho_AB = rho_g(4,0)
rho_AB

Matrix([
[rho_00, rho_01, rho_02, rho_03],
[rho_04, rho_05, rho_06, rho_07],
[rho_08, rho_09, rho_10, rho_11],
[rho_12, rho_13, rho_14, rho_15]])

In [171]:
rhokb_AB = m_braket(rho_AB)
rhokb_AB

rho_00*|00>*<00| + rho_01*|00>*<01| + rho_02*|00>*<10| + rho_03*|00>*<11| + rho_04*|01>*<00| + rho_05*|01>*<01| + rho_06*|01>*<10| + rho_07*|01>*<11| + rho_08*|10>*<00| + rho_09*|10>*<01| + rho_10*|10>*<10| + rho_11*|10>*<11| + rho_12*|11>*<00| + rho_13*|11>*<01| + rho_14*|11>*<10| + rho_15*|11>*<11|

#### Matriz densidade reduzida do subsistema A - Traço parcial no subsistema B

In [172]:
rho_A = ptraceB(2,2,rho_AB)
rho_A

Matrix([
[rho_00 + rho_05, rho_02 + rho_07],
[rho_08 + rho_13, rho_10 + rho_15]])

In [173]:
rhokb_A = m_braket(rho_A)
rhokb_A

(rho_00 + rho_05)*|0>*<0| + (rho_02 + rho_07)*|0>*<1| + (rho_08 + rho_13)*|1>*<0| + (rho_10 + rho_15)*|1>*<1|

#### Matriz densidade reduzida do subsistema B - Traço parcial no subsistema A

In [174]:
rho_B = ptraceA(2,2,rho_AB)
rho_B

Matrix([
[rho_00 + rho_10, rho_01 + rho_11],
[rho_04 + rho_14, rho_05 + rho_15]])

In [175]:
rhokb_B = m_braket(rho_B)
rhokb_B

(rho_00 + rho_10)*|0>*<0| + (rho_01 + rho_11)*|0>*<1| + (rho_04 + rho_14)*|1>*<0| + (rho_05 + rho_15)*|1>*<1|

### Exemplo 3 - Base de Bell

Base de Bell - $\Phi_+$

In [176]:
Phi_p = (tp(cb(2,0),cb(2,0)) + tp(cb(2,1),cb(2,1)))/sqrt(2)
Phi_p

Matrix([
[sqrt(2)/2],
[        0],
[        0],
[sqrt(2)/2]])

In [177]:
Phikb_p = m_braket(Phi_p)
simplify(Phikb_p)

sqrt(2)*(|00> + |11>)/2

In [178]:
rho_Phi_p = proj(Phi_p)
rho_Phi_p

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [179]:
rhokb_Phi_p = m_braket(proj(Phi_p))
rhokb_Phi_p.simplify()

(|00>*<00| + |00>*<11| + |11>*<00| + |11>*<11|)/2

Base de Bell - $\Psi_+$

In [180]:
Psi_p = (tp(cb(2,0),cb(2,1)) + tp(cb(2,1),cb(2,0)))/sqrt(2)
Psi_p

Matrix([
[        0],
[sqrt(2)/2],
[sqrt(2)/2],
[        0]])

In [181]:
Psikb_p = m_braket(Psi_p)
Psikb_p.simplify()

sqrt(2)*(|01> + |10>)/2

In [182]:
rho_Psi_p = proj(Psi_p)
rho_Psi_p

Matrix([
[0,   0,   0, 0],
[0, 1/2, 1/2, 0],
[0, 1/2, 1/2, 0],
[0,   0,   0, 0]])

In [183]:
rhokb_Psi_p = m_braket(proj(Psi_p))
rhokb_Psi_p.simplify()

(|01>*<01| + |01>*<10| + |10>*<01| + |10>*<10|)/2

Base de Bell - $\Psi_-$

In [184]:
Psi_m = (tp(cb(2,0),cb(2,1)) - tp(cb(2,1),cb(2,0)))/sqrt(2)
Psi_m

Matrix([
[         0],
[ sqrt(2)/2],
[-sqrt(2)/2],
[         0]])

In [185]:
Psikb_m = m_braket(Psi_m)
Psikb_m.simplify()

sqrt(2)*(|01> - |10>)/2

In [186]:
rho_Psi_m = proj(Psi_m)
rho_Psi_m

Matrix([
[0,    0,    0, 0],
[0,  1/2, -1/2, 0],
[0, -1/2,  1/2, 0],
[0,    0,    0, 0]])

In [187]:
rhokb_Psi_m = m_braket(proj(Psi_m))
rhokb_Psi_m.simplify()

(|01>*<01| - |01>*<10| - |10>*<01| + |10>*<10|)/2

Base de Bell - $\Phi_-$

In [188]:
Phi_m = (tp(cb(2,0),cb(2,0)) - tp(cb(2,1),cb(2,1)))/sqrt(2)
Phi_m

Matrix([
[ sqrt(2)/2],
[         0],
[         0],
[-sqrt(2)/2]])

In [189]:
Phikb_m = m_braket(Phi_m)
Phikb_m.simplify()

sqrt(2)*(|00> - |11>)/2

In [190]:
rho_Phi_m = proj(Phi_m)
rho_Phi_m

Matrix([
[ 1/2, 0, 0, -1/2],
[   0, 0, 0,    0],
[   0, 0, 0,    0],
[-1/2, 0, 0,  1/2]])

In [191]:
rhokb_Phi_m = m_braket(proj(Phi_m))
rhokb_Phi_m.simplify()

(|00>*<00| - |00>*<11| - |11>*<00| + |11>*<11|)/2

# Outras funções

## Projective measure

In [201]:
# Projective measure: subsistemas ABCD para sobrar apenas o D
#
# IMPORTANTE: D subsystem on the left
#
# rho_AB
#
# rho_B_red = Proj(A) * rho_AB * Proj(A)
#
# seq_bin:
# If you want project |101>_ABC of |101>_ABC(|0>_D+|1>_D) send seq_bin = '101'
# The last qubit is the susystem that you want to keep
#
# db = dimension expected to the matrix B (ex above is 2 -> M_{2x2}, only D system)
# because we are going to remove the ABC systems
#
def projM(db, seq_bin, rho): # seq_bin 
    rhoB = zeros(db,db)
    dn= int(seq_bin+str(0), 2) # num decimal number of seq_bin
    #print(dn)
    for j in range(0, db):
        for k in range(0, db):
                rhoB[j,k] = rho[j+dn,k+dn]
    return rhoB/tr(rhoB)

In [202]:
rho_AB = rho_g(4,0)
rho_AB

Matrix([
[rho_00, rho_01, rho_02, rho_03],
[rho_04, rho_05, rho_06, rho_07],
[rho_08, rho_09, rho_10, rho_11],
[rho_12, rho_13, rho_14, rho_15]])

In [203]:
m_braket_g(2,rho_AB)

rho_00*|00>*<00| + rho_01*|00>*<01| + rho_02*|00>*<10| + rho_03*|00>*<11| + rho_04*|01>*<00| + rho_05*|01>*<01| + rho_06*|01>*<10| + rho_07*|01>*<11| + rho_08*|10>*<00| + rho_09*|10>*<01| + rho_10*|10>*<10| + rho_11*|10>*<11| + rho_12*|11>*<00| + rho_13*|11>*<01| + rho_14*|11>*<10| + rho_15*|11>*<11|

In [204]:
rho_0B = projM(2,'0', rho_0)
rho_0B

Matrix([
[rho_00/(rho_00 + rho_06), rho_01/(rho_00 + rho_06)],
[rho_05/(rho_00 + rho_06), rho_06/(rho_00 + rho_06)]])

In [205]:
simplify(m_braket(rho_0B))

(rho_00*|0>*<0| + rho_01*|0>*<1| + rho_05*|1>*<0| + rho_06*|1>*<1|)/(rho_00 + rho_06)

In [206]:
rho_1B = projM(2,'1',rho_0)
rho_1B

Matrix([
[rho_12/(rho_12 + rho_18), rho_13/(rho_12 + rho_18)],
[rho_17/(rho_12 + rho_18), rho_18/(rho_12 + rho_18)]])

In [207]:
simplify(m_braket(rho_1B))

(rho_12*|0>*<0| + rho_13*|0>*<1| + rho_17*|1>*<0| + rho_18*|1>*<1|)/(rho_12 + rho_18)

## Dephasing map

In [208]:
# Essa função aplica o dephasing map M_j em 2 qubits, tanto no W como no Z
# 
# M₀=|00><00|=(|0>⊗|0>)(<0|⊗<0|)=[ 1	0	0	0
#                                   0	0	0	0
#                                   0	0	0	0
#                                   0	0	0	0]
def dephasing_map_2qubits(rho):
    M0,M1,M2,M3 = zeros(4, 4), zeros(4, 4), zeros(4, 4), zeros(4, 4)
    M0[0, 0] = 1
    M1[1, 1] = 1
    M2[2, 2] = 1
    M3[3, 3] = 1
    Phi_rho = M0*rho*M0 + M1*rho*M1 + M2*rho*M2 + M3*rho*M3
    return Phi_rho

In [213]:
rho_0 = rho_g(4,0); rho_0

Matrix([
[rho_00, rho_01, rho_02, rho_03],
[rho_04, rho_05, rho_06, rho_07],
[rho_08, rho_09, rho_10, rho_11],
[rho_12, rho_13, rho_14, rho_15]])

In [216]:
m_braket(rho_0)

rho_00*|00>*<00| + rho_01*|00>*<01| + rho_02*|00>*<10| + rho_03*|00>*<11| + rho_04*|01>*<00| + rho_05*|01>*<01| + rho_06*|01>*<10| + rho_07*|01>*<11| + rho_08*|10>*<00| + rho_09*|10>*<01| + rho_10*|10>*<10| + rho_11*|10>*<11| + rho_12*|11>*<00| + rho_13*|11>*<01| + rho_14*|11>*<10| + rho_15*|11>*<11|

In [217]:
dephasing_map_2qubits(rho_0)

Matrix([
[rho_00,      0,      0,      0],
[     0, rho_05,      0,      0],
[     0,      0, rho_10,      0],
[     0,      0,      0, rho_15]])

In [218]:
m_braket(dephasing_map_2qubits(rho_0))

rho_00*|00>*<00| + rho_05*|01>*<01| + rho_10*|10>*<10| + rho_15*|11>*<11|

In [219]:
def id_sympy(n):
    identity = sym.eye(n)
    return identity

In [220]:
# Essa função aplica o dephasing map M0 = [1,0],[0,0] e M1 = [0,0],[0,1] somente no primeiro qubit
# 
def dephasing_map_2qubits_primeiro(rho):
    M0,M1 = zeros(2, 2), zeros(2, 2)
    M0[0, 0] = 1
    M0_2 = tp(M0,  id_sympy(2))
    M1[1, 1] = 1
    M1_2 = tp(M1,  id_sympy(2))
    Phi_rho = M0_2*rho*M0_2 + M1_2*rho*M1_2
    return Phi_rho

In [224]:
m_braket(rho_0)

rho_00*|00>*<00| + rho_01*|00>*<01| + rho_02*|00>*<10| + rho_03*|00>*<11| + rho_04*|01>*<00| + rho_05*|01>*<01| + rho_06*|01>*<10| + rho_07*|01>*<11| + rho_08*|10>*<00| + rho_09*|10>*<01| + rho_10*|10>*<10| + rho_11*|10>*<11| + rho_12*|11>*<00| + rho_13*|11>*<01| + rho_14*|11>*<10| + rho_15*|11>*<11|

In [226]:
dephasing_map_2qubits_primeiro(rho_0)

Matrix([
[rho_00, rho_01,      0,      0],
[rho_04, rho_05,      0,      0],
[     0,      0, rho_10, rho_11],
[     0,      0, rho_14, rho_15]])

In [228]:
m_braket(dephasing_map_2qubits_primeiro(rho_0))

rho_00*|00>*<00| + rho_01*|00>*<01| + rho_04*|01>*<00| + rho_05*|01>*<01| + rho_10*|10>*<10| + rho_11*|10>*<11| + rho_14*|11>*<10| + rho_15*|11>*<11|